# Cashflow Definition
Cashflow Defintion query takes as input a string identifying the pillar: IFRS 17, Solvency, ...

This query has the default logic necessary to provide a fall back (Best Estimate) in case of Scenario calculation. 

In [0]:
// Should be required only to compile this NB in isolation (debugging)
#!import "../Constants/Ifrs17CalculationEngine"

In [0]:
#!import "../DataModel/DataStructure"

In [0]:
public static async Task<T[]> QueryCashflowAsync<T,P>(this IWorkspace workspace, IDataSource dataSource, 
                                                                      Guid targetPartition, Guid defaultPartition,
                                                                      ImportArgs args,
                                                                      string pillar) //pillar should be contained in the args(.ImportFormat?)
    where T : IPartitioned, IWithPillar
    where P : IPartition
{
    //FilterExpression
    Expression<Func<T, bool>> filterExpression = x => x.Pillar == pillar;
    
    var isRelaxed = args.Scenario == null;

    var variablesFromWorkspace = await workspace.LoadPartitionedDataAsync<T,P>(targetPartition, filterExpression);
    if( !isRelaxed || variablesFromWorkspace.Any())
        return variablesFromWorkspace;
    
    var variablesFromDataSource = await dataSource.LoadPartitionedDataAsync<T,P>(targetPartition, filterExpression);
    if(variablesFromDataSource.Any()) return variablesFromDataSource;

    //var variablesFromDataSource = await dataSource.LoadPartitionedDataAsync<T,P>(defaultPartition, x => x.Pillar == pillar);

    return variablesFromWorkspace.Union(dataSource.LoadPartitionedDataAsync<T,P>(defaultPartition, x => x.Pillar == pillar), EqualityComparer<T>.Instance).ToArray();
}